# Лабораторная работа №1

Класс игры

In [1]:
import torch

class KalahaState():

	def __init__(self):
		#Доска предстаёт в таком виде для того, чтобы её можно было "переворачивать", когда ходит другой игрок
		self.board = torch.tensor([6,6,6,6,6,6,0,0,6,6,6,6,6,6]) + 0.
		self.view_board = self.board.clone()
		self.prev_player = 0 #Предыдущий игрок
		self.cur_player = 0 #Текущий игрок
		self.INDEXS = [0,1,2,3,4,5,6,7,13,12,11,10,9,8] #Индексы для элементов доски

	def move(self, point):
		if (self.prev_player != self.cur_player):
			self.board = torch.flip(self.board, [0]) #Если игрок сменился, переворачиваем доску
		
		count_stone = int(self.board[point])
		self.board[point] = 0
		point += 1
		last_point = 6
		while (count_stone > 0):
			last_point = point
			if point == 7:
				point += 1
			self.board[self.INDEXS[point]] += 1
			point += 1
			point %= 14
			count_stone -= 1

#Закончили на пустой ямке
		if (self.board[last_point] == 1 and (last_point >= 0 and last_point <= 5) and self.board[8 + last_point] != 0):
			self.board[6] += self.board[8 + last_point] + self.board[last_point]
			self.board[8 + last_point] = 0
			self.board[last_point] = 0

#Если закончились камни
		if (torch.sum(self.board[0:6]) == 0):
			self.board[7] += torch.sum(self.board[8:14])
			self.board[8:14] = 0

		if (torch.sum(self.board[8:14]) == 0):
			self.board[6] += torch.sum(self.board[0:6])
			self.board[0:6] = 0

		if (0 != self.cur_player):
			self.view_board = torch.flip(self.board.clone(), [0])
		else:
			self.view_board = self.board.clone()

#Победа игроков
		if (self.board[6] > 36):
			return self.cur_player

		if (self.board[7] > 36):
			return (self.cur_player + 1) % 2

#Ничья
		if (self.board[6] == 36 and self.board[7] == 36):
			return 2

#Конец хода
		if (last_point != 6):
			self.prev_player = self.cur_player
			self.cur_player = (self.cur_player + 1) % 2
		else:
			self.prev_player = self.cur_player
		return -1

Создание сети

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import datetime
import random
import os.path

#Выбираем ход
def moveAgent(brd):
	k = 0
	while k == 0:
		pos = random.randint(0, 5)
		k = int(brd[8 + pos])
	return 5 - pos

#Создаём модель
model = torch.nn.Sequential(
        torch.nn.Linear(14,20), 
        torch.nn.Sigmoid(),
        torch.nn.Linear(20,30),
        torch.nn.PReLU(),
        torch.nn.Linear(30, 6), 
        torch.nn.Softmax()
        )

Обучение сети и проверка

In [6]:
#Обучение будет происходить в цикле, чтобы не запускать его по кругу множество раз, а просто поменять 1 на 10000
iny = 0
while iny < 1:

	iny += 1
	if (os.path.exists('hkk.pt')):
		model = torch.load('hkk.pt') #Загружаем сеть
		print('re:')
	
	optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)
	
	count_games = 1000
	wins = 0
	draws = 0
	loses = 0
	alpha = 0.7
	errors = 0
	for game in range(count_games):
		kalaha = KalahaState()
		condition = -1
		loss = 0
		while condition == -1:
			if (kalaha.cur_player != 0):
				condition = kalaha.move(moveAgent(kalaha.view_board))
			else:
				probs = model(kalaha.view_board)
				mod = Categorical(probs)
				act = mod.sample()
				if (kalaha.view_board[int(act)] == 0): #Ошибка
					errors += 1
					reward = -100000
					loss = -alpha*reward*mod.log_prob(act)
					optimizer.zero_grad()
					loss.backward()
					optimizer.step()
				condition = kalaha.move(int(act))
		if condition == 0:
			wins+= 1
		if condition == 1:
			loses+= 1
		if condition == 2:
			draws+= 1

	torch.save(model, 'hkk.pt') #Сохраняем сеть

print('Ошибок: ', errors)
print('Побед: ', wins)
print('Ничьих: ', draws)   
print('Поражений: ', loses)      


re:
Ошибок:  0
Побед:  579
Ничьих:  62
Поражений:  359
